In [3]:
import pandas as pd

file_txt = "mapa.TXT"

colunas =["Lote", "N Pedido", "Produto", "Cor Etiq", "QTD Vol", "Data Corte", "Data Emb", "Status", "N/A"]
df = pd.read_csv(file_txt, skiprows=23, delimiter=";", names=colunas)
df = df.dropna(subset=["N Pedido"])
df = df.drop(["QTD Vol", "Data Corte","Status", "N/A"], axis=1)
df ["Isopor"] = "Não Conferido"
df ["Tecnoplac"] = "Não Conferido"
df ["Lote_Cx"] = "Não Conferido"
df.to_excel("Mapa_Almoxarifado.xlsx",index=False,engine='openpyxl')


In [4]:
import openpyxl
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.styles import Border, Side, Alignment, Font, PatternFill
import pandas as pd
import subprocess

wb = openpyxl.load_workbook("Mapa_Almoxarifado.xlsx")
print(wb)
sheet = wb.active

validacao = '"Mapeado, Parcial,  Conferido, Não Conferido, Separado, Entregue"'

rule = DataValidation(type='list',formula1=validacao, allow_blank=True)

rule.error  = "Entrada Inválida"
rule.errorTitle = "Selecione Corretamente"

rule.prompt = "Selecione a Lista"
rule.promptTitle = "Selecione a opção"

sheet.add_data_validation(rule)

rule.add("F2:H250")

# Adicione bordas de separação para linhas e colunas
borda = Border(left=Side(style='thin'), 
                        right=Side(style='thin'), 
                        top=Side(style='thin'), 
                        bottom=Side(style='thin'))

for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
    for cell in row:
        cell.border = borda

# Definindo o tamanho das colunas
column_widths = {
        'A': 9,  # Lote
        'B': 11,  # N Pedido
        'C': 60,  # Produto
        'D': 11,  # Cor
        'E': 20,  # Data Embalagem
        'F': 15,  # Isopor
        'G': 15,  # Tecnoplac
        'H': 15,  # Lote_Caixinha
}

for coluna, tamanho in column_widths.items():
    sheet.column_dimensions[coluna].width = tamanho

for row in sheet[2:sheet.max_row]:
    for col1 in [0,1, 3, 4, 5,6,7]:
        cell = row[col1]
        cell.alignment = Alignment(horizontal='center')

 # Define a fonte com tamanho de 10 pontos
fonte_padrao = Font(size=10)

# Aplica a fonte para todas as células
for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
    for cell in row:
        cell.font = fonte_padrao


# Estilo do cabeçalho (A1:H1)
fonte_cabecalho = Font(size=11, bold=True, color="FFFFFF")
fundo_roxo_claro = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")


for cell in sheet["A1:H1"][0]:
    cell.font = fonte_cabecalho
    cell.fill = fundo_roxo_claro
    cell.alignment = Alignment(horizontal="center", vertical="center")
    
# Define os preenchimentos
cinza_escuro = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")  # DarkGray
cinza_claro  = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")  # LightGray

# Aplica cores alternadas nas linhas (ignorando o cabeçalho, começa na linha 2)
for i, row in enumerate(sheet.iter_rows(min_row=2, max_row=sheet.max_row), start=0):
    fill = cinza_escuro if i % 2 == 0 else cinza_claro
    for cell in row:
        cell.fill = fill

sheet.freeze_panes = "A2"

# Salva o novo arquivo
wb.save("Mapa_Almoxarifado.xlsx")

subprocess.Popen(['start', 'excel.exe', 'Mapa_Almoxarifado.xlsx'], shell=True)


<Popen: returncode: None args: ['start', 'excel.exe', 'Mapa_Almoxarifado.xlsx']>